In [2]:
import numpy as np
from scipy.stats import norm
from scipy.stats import multivariate_normal
data = np.array([
    [0.6, 0, 0.2, 0.4],
    [0.1, 1, -0.1, -0.4],
    [0.2, 0, -0.1, 0.2],
    [0.1, 2, 0.8, 0.8],
    [0.3, 1, 0.1, 0.3],
    [-0.1, 2, 0.2, -0.2],
    [-0.3, 2, -0.1, 0.2],
    [0.2, 1, 0.5, 0.6],
    [0.4, 0, -0.4, -0.7],
    [-0.2, 2, 0.4, 0.3]
])
target = np.array([0, 0, 0, 0, 1, 1, 1, 1, 1, 1])

In [3]:
c0_filter = []
c1_filter = []
for el in target:
    if el == 0:
        c1_filter += [False]
        c0_filter += [True]
    else:
        c1_filter += [True]
        c0_filter += [False]

In [4]:
p_c0 = np.count_nonzero(target == 0)/target.size
p_c1 = 1 - p_c0

In [5]:
y1_c0_data = data[:, 0][c0_filter]
y1_c0_mean = y1_c0_data.mean()
y1_c0_std = y1_c0_data.std(ddof=1)
y1_c1_data = data[:, 0][c1_filter]
y1_c1_mean = y1_c1_data.mean()
y1_c1_std = y1_c1_data.std(ddof=1)
print("c = 0 ", "mean ", y1_c0_mean, " std ", y1_c0_std)
print("c = 1 ", "mean ", y1_c1_mean, " std ", y1_c1_std)

c = 0  mean  0.24999999999999997  std  0.23804761428476162
c = 1  mean  0.049999999999999996  std  0.28809720581775866


In [6]:
y2_c0_data = data[:, 1][c0_filter]
y2_c1_data = data[:, 1][c1_filter]
p_A_c0 = np.count_nonzero(y2_c0_data == 0)/y2_c0_data.size
p_A_c1 = np.count_nonzero(y2_c1_data == 0)/y2_c1_data.size
p_B_c0 = np.count_nonzero(y2_c0_data == 1)/y2_c0_data.size
p_B_c1 = np.count_nonzero(y2_c1_data == 1)/y2_c1_data.size
p_C_c0 = np.count_nonzero(y2_c0_data == 2)/y2_c0_data.size
p_C_c1 = np.count_nonzero(y2_c1_data == 2)/y2_c1_data.size
p_y2_table = np.array([
    [p_A_c0, p_A_c1],
    [p_B_c0, p_B_c1],
    [p_C_c0, p_C_c1]
])  

In [7]:
cov_matrix_y34_c0 = np.cov(data[:,2:4][c0_filter].T)
inv_cov_matrix_y34_c0 = np.round(np.linalg.inv(cov_matrix_y34_c0), decimals=2)
y34_c0_mean = (data[:,2:4][c0_filter]).mean(axis=0)

cov_matrix_y34_c1 = np.round(np.cov(data[:,2:4][c1_filter].T), decimals=3)
inv_cov_matrix_y34_c1 = np.round(np.linalg.inv(cov_matrix_y34_c1),decimals=2)
y34_c1_mean = np.round(data[:,2:4][c1_filter].mean(axis=0), decimals=3)

print("c = 0 ", "mean ", y34_c0_mean, "\n Covariance Matrix \n", cov_matrix_y34_c0, "\n Inverse \n", inv_cov_matrix_y34_c0)
print("c = 1 ", "mean ", y34_c1_mean, "\n Covariance Matrix \n", cov_matrix_y34_c1, "\n Inverse \n", inv_cov_matrix_y34_c1)

c = 0  mean  [0.2  0.25] 
 Covariance Matrix 
 [[0.18 0.18]
 [0.18 0.25]] 
 Inverse 
 [[ 19.84 -14.29]
 [-14.29  14.29]]
c = 1  mean  [0.117 0.083] 
 Covariance Matrix 
 [[0.11  0.122]
 [0.122 0.214]] 
 Inverse 
 [[ 24.72 -14.09]
 [-14.09  12.71]]


In [44]:
def predictClass(x_new, treshold, c1_values):
    p_c0_xnew1 = np.round(norm.pdf(x_new[0], y1_c0_mean, y1_c0_std), decimals=5)
    p_c1_xnew1 = np.round(norm.pdf(x_new[0], y1_c1_mean, y1_c1_std), decimals=5)
    p_c0_xnew34 = np.round(multivariate_normal.pdf(x_new[2:], mean=y34_c0_mean, cov=inv_cov_matrix_y34_c0), decimals=5)
    p_c1_xnew34 = np.round(multivariate_normal.pdf(x_new[2:], mean=y34_c1_mean, cov=inv_cov_matrix_y34_c1), decimals=5)
    p_c0_xnew = p_c0_xnew1*p_y2_table[int(x_new[1])][0]*p_c0_xnew34*p_c0
    p_c1_xnew = p_c1_xnew1*p_y2_table[int(x_new[1])][1]*p_c1_xnew34*p_c1
    c1_percent = p_c1_xnew/(p_c1_xnew+p_c0_xnew)*100
    c1_values.append(c1_percent)
    #print(p_c0_xnew1, p_y2_table[int(x_new[1])][0], np.round(p_c0_xnew34,decimals=5))
    #print("p(y_1=",x_new[0],"| c=0)=",p_c0_xnew1,"   p(y_2=",x_new[1],"| c=0)=", 0,"   p(y_3=",x_new[2],", y_4=",x_new[3],"| c=0)=",p_c0_xnew34) 
    #print("p(y_1=",x_new[0],"| c=1)=",p_c1_xnew1,"   p(y_2=",x_new[1],"| c=1)=", 0,"   p(y_3=",x_new[2],", y_4=",x_new[3],"| c=1)=",p_c1_xnew34) 
    #print(np.round(p_c0_xnew, decimals=5), np.round(p_c1_xnew,decimals=5))
    #print(np.round(c1_percent, decimals=1))
    
    if c1_percent > treshold:
        return 1
    else: 
        return 0

def F1_score(confusion_matrix):
    precision = confusion_matrix[0][0]/np.sum(confusion_matrix[0,:])
    recall = confusion_matrix[0][0]/np.sum(confusion_matrix[:,0])
    return 2*1/(1/precision+1/recall)

In [52]:

treshold = 15
def NB(treshold):
    number_tests = 0
    correct_tests = 0
    confusion_matrix = np.array([[0,0],[0,0]])
    c1_values= []
    for i in range(len(data)):
        number_tests += 1
        if predictClass(data[i], treshold, c1_values) == target[i]:
            correct_tests += 1
            if target[i] == 0: confusion_matrix[1][1] += 1
            else: confusion_matrix[0][0] += 1
            #print("Correct!", target[i])
            continue
        if target[i] == 0: confusion_matrix[0][1] += 1
        else: confusion_matrix[1][0] += 1
        #print("Wrong!", (target[i]+1)%2 ,"and answer is", target[i])
    accuracy = correct_tests/number_tests*100
    #print(accuracy, "%")
    #print(confusion_matrix)
    #print(F1_score(confusion_matrix))
    c1_values = sorted(c1_values)
    return accuracy, c1_values
    
c1_values = NB(10)[1]

In [73]:
import math
x_c1_values = []
y_c1_values = []
for el in c1_values:
    x_c1_values.append(math.floor(el))
x_c1_values.append(99)
for el in x_c1_values:
    y_c1_values.append(int(NB(el)[0]))
print("c1_v ", np.round(c1_values,decimals=2))
print("value", x_c1_values)
print("acc  ", y_c1_values)

c1_v  [13.88 18.33 23.58 48.87 54.54 63.48 70.92 84.34 89.27 93.46]
value [13, 18, 23, 48, 54, 63, 70, 84, 89, 93, 99]
acc   [60, 70, 60, 70, 60, 50, 60, 70, 60, 50, 40]
